In [9]:
# clean data, create embeddings and upload to pinecone

import os
import pandas as pd
from dotenv import load_dotenv
import sys
import openai
import pinecone
from tqdm import tqdm
import math
import requests
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
import re
from collections import Counter
from itertools import chain

load_dotenv()

True

In [10]:
# set constants
EMBEDDINGS_MODEL = "text-embedding-ada-002"
EMBEDDINGS_DIMENSION = 1536
EMBEDDINGS_MAX_TOKENS = 8191
COMPLETION_MODEL = "text-davinci-003"
DAVINCI_MAX_TOKENS = 4096

PINECONE_BATCH_SIZE = 32

In [11]:
openai.api_key = os.getenv("OPENAI_API_KEY")
pinecone.init(
    api_key=os.environ.get('PINECONE_API_KEY'),
    environment="us-west1-gcp"
)
DATABASE_URL = os.getenv("DATABASE_URL")
%load_ext sql
%sql $DATABASE_URL
engine = create_engine(DATABASE_URL)

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [12]:
# 1000 tokens ~ 750 words; there is no way to get the number of tokens from the API for 2nd gen models for now
# 1 token ~ 4 characters; ceil to be safe
def token_estimate(text):
    return int(math.ceil(len(text) / 4))

# we know that openai ada model costs $0.0004 / 1K tokens
def cost_estimate(tokens):
    return tokens / 1000 * 0.0004

# get embeddings for text
def get_embedding(text: str) -> list[float]:
    result = openai.Embedding.create(
        model=EMBEDDINGS_MODEL,
        input=text
    )
    return result["data"][0]["embedding"]

def get_pinecone_index(index_name: str) -> pinecone.Index:
    return pinecone.Index(index_name)

def set_embedded_status(df: pd.DataFrame):
    # the rows that have will be added to pinecone index, set their embedded status to 1 in database
    df['embedded'] = 1
    df.to_sql('crawl_data', engine, if_exists='replace', index=False)

def add_to_pinecone(df: pd.DataFrame, index: pinecone.Index):
    # get embeddings for each row and add to pinecone index
    for i in tqdm(range(0, df.shape[0], PINECONE_BATCH_SIZE)):
        # set end position of batch
        i_end = min(i+PINECONE_BATCH_SIZE, df.shape[0])
        ids_batch = [str(n) for n in range(i, i_end)]
        metadata_batch = df.iloc[i: i_end].to_dict('records')
        # create embeddings for the data column in df
        embeds = [get_embedding(text) for text in df.iloc[i: i_end]['data']]
        to_upsert = zip(ids_batch, embeds, metadata_batch)
        index.upsert(vectors=list(to_upsert))


In [13]:
def cleanup(df: pd.DataFrame) -> pd.DataFrame:
    # convert data column into list of p texts. data is a list of p tags but when u read from db, it is a string
    df['data'] = df['data'].apply(lambda x: re.findall(r'\"(.*?)\"', x))

    # remove duplicate urls
    old_rows = df.shape[0]
    df = df.drop_duplicates(subset=['url'], keep='first')
    print('duplicate urls deleted:', old_rows - df.shape[0])

    # delete duplicate titles
    old_rows = df.shape[0]
    df = df.drop_duplicates(subset=['title'], keep='first')
    print('duplicate titles deleted:', old_rows - df.shape[0])

    # remove most common words as they are not informative
    # flatten list of lists
    data = list(chain.from_iterable(df['data']))
    # get most common words, returns list of tuples
    most_common = Counter(data).most_common(10)
    # convert to list of words
    most_common = [x[0] for x in most_common]
    # remove 10 most common words from lists of data
    df['data'] = df['data'].apply(lambda x: [word for word in x if word not in most_common])

    # remove empty data
    old_rows = df.shape[0]
    df = df[df['data'].apply(lambda x: len(x) > 0)]
    print('empty data deleted:', old_rows - df.shape[0])

    # concat data into string
    df['data'] = df['data'].apply(lambda x: ' '.join(x))

    # add token estimate column
    df['token_estimate'] = df['data'].apply(lambda x: token_estimate(x))
    # davinci token limit prompt + completion is 4096
    # so we will keep our request token to 2500 (excluding prompt suffix since its tiny)
    # if there is anything above 2500 tokens we will get rid of it for now
    # we will also get rid of anything below 20 tokens

    # remove rows with token estimate above 2500
    old_rows = df.shape[0]
    df = df[df['token_estimate'] < 2500]
    print('rows deleted token estimate above 2500', old_rows - df.shape[0])

    # remove rows with token estimate below 20
    old_rows = df.shape[0]
    df = df[df['token_estimate'] > 20]
    print('rows deleted token estimate below 20', old_rows - df.shape[0])

    # metadata size limit is 10KB for pinecone
    # so we will remove rows with metadata size above 10240 bytes
    df['metadata_size'] = df.apply(lambda x: sys.getsizeof(x.to_json()), axis=1)
    # remove rows with metadata size above 10240
    old_rows = df.shape[0]
    df = df[df['metadata_size'] < 10240]
    # remove metadata size column
    df = df.drop(columns=['metadata_size'])
    return df

In [ ]:
# I store crawl data in a postgres db, columns: id:int, url:text, title:text, data:text(list of p tags text), company_id:int(what company ur indexed data belongs to)
# there is another table called company which has id:int, name:text (also the pinceone index name)

COMPANY_ID = 1

# get index name from company table
index_name = engine.execute('SELECT * FROM company')
# get name where id is COMPANY_ID
index_name = [x[1] for x in index_name if x[0] == COMPANY_ID][0]
index = get_pinecone_index(index_name)

# get all rows that are not embedded
df = pd.read_sql(f'SELECT * FROM crawl_data WHERE embedded = 0 AND company_id = {COMPANY_ID}', engine)
df = cleanup(df)
add_to_pinecone(df, index)
set_embedded_status(df)